## Import Statements

In [1]:
%load_ext autoreload
%autoreload 2

from matplotlib import gridspec, rc
from matplotlib import ticker as mticker
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.colors as colors

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy
from numpy.random.mtrand import beta
from sklearn.preprocessing import normalize


from subpop import *
from learner_utils import *

%matplotlib inline

np.set_printoptions(precision=3, suppress=True)

In [2]:
## Plotting settings
rc('text', usetex = False)
rc('font', family = 'serif')
TITLE_SIZE = 13
LABEL_SIZE = 11
LEGEND_TITLE_SIZE = 12
LEGEND_SIZE = 11
TICK_SIZE = 11
FONT = 'serif'
params = {}
params['legend.title_fontsize'] = LEGEND_TITLE_SIZE
params['axes.labelsize'] = LABEL_SIZE
params['axes.titlesize'] = TITLE_SIZE
params['legend.fontsize'] = LEGEND_SIZE
params["xtick.labelsize"]= TICK_SIZE
params["ytick.labelsize"] = TICK_SIZE
params["font.family"] = "Times New Roman"
context = sns.plotting_context("paper", rc=params)
sns.set_theme(style="whitegrid", font=FONT)

## Dynamics Simulation

In [3]:
def run_experiment_with_prices(T, subpops, min_fn=quadratic_min, early_stop=False, verbose = True):
    n_learners = len(subpops[0].alphas)
    if verbose:
        print("Initial Conditions:")
        print(f"There are {n_learners} initial learners")
        print(f"Subpopulation splits: {[s.beta for s in subpops]}")
        print(f"Initial allocations: {[s.alphas for s in subpops]}")
        print(f"Optimal decisions theta for each subpop (row-wise) {[s.phi for s in subpops]}")
    average_risks_subpop = []
    average_risks_learner = []
    all_risks = []
    all_thetas = []
    all_alphas = []
    all_prices = []
    all_profits = []
    for t in range(T):
        if t == 0:
            prices = np.zeros(n_learners)
            thetas = np.array(learner_decisions(subpops, min_fn=min_fn))
        else:
            thetas = learner_decisions(subpops, min_fn=min_fn)
            #thetas, prices = learner_price_decisions(thetas, prices, subpops, min_fn=min_fn)
            thetas = np.array(thetas)
            prices = np.array(prices)
        alpha = subpop_decisions(thetas, subpops, epsilon=0.1, prices=prices)
        prices = learner_max_price(thetas, prices, subpops, min_fn=min_fn)
        if t < T*1/2:
            prices[0] = 0
            #prices = np.zeros_like(prices)
        alpha = subpop_decisions(thetas, subpops, epsilon=0.1, prices=prices)
        all_thetas.append(thetas)
        all_prices.append(prices)
        all_alphas.append(alpha.T)
        all_profits.append(prices * (alpha.sum(0)))
        risks = get_all_risks(thetas, subpops)
        all_risks.append(risks)
        a_risk_subpop = average_risk_subpop(thetas, subpops, prices=prices)
        average_risks_subpop.append(a_risk_subpop)
        a_risk_learner = average_risk_learner(thetas, subpops)
        average_risks_learner.append(a_risk_learner)
        #print(f"t = {t}; Prices: {prices}")
        print('-' * 10)
        print(f't = {t}')
        print(f'p: {prices}')
        print(f'α: {alpha}')
        print(f'θ: {thetas}')
        
        if early_stop:
            c_counts = convergent_count(subpops)
            if min(c_counts) > 20:
                break
    if verbose:
        print("Final Conditions:")
        print(f'\t\tLearners decisions: {thetas}')
        print(f'\t\tSubpopulation allocations: {alpha}')
    return(average_risks_subpop, average_risks_learner, all_risks, all_thetas, all_alphas, all_prices, all_profits) 

## Figure 1: Risk profiles over time

In [4]:
def run_trial(allocation=None, min_fn = quadratic_min):
    T = 100
    # relative sizes
    beta_1 = 1
    beta_2 = 1 
    beta_3 = 1
    # initial allocations
    if allocation is  None:
        dist = np.array([[0.01, 0.02, -0.04], [-0.01,-0.01, 0.02]])
        dist = np.random.rand(2,3)*0.1
        alpha = np.array([[1,1, 1], [1,1, 1]]) + dist
        # alpha_ij <- fraction of subpop j going to learner i
        alpha =  normalize(alpha, axis=0, norm='l1')   
    else:
        alpha = allocation
    # optimal thetas for each subpop
    phi_1 = np.array([1, 0, 0])
    phi_2 = np.array([0, 1, 0])
    phi_3 = np.array([0, 0, 1])
    # phi = np.array([phi_1, phi_2])

    pop1 = QuadraticSubPop(phi_1, beta_1, alpha[:,0], price_sensitivity=1)
    pop2 = QuadraticSubPop(phi_2, beta_2, alpha[:,1], price_sensitivity=1)
    pop3 = QuadraticSubPop(phi_3, beta_3, alpha[:,2], price_sensitivity=1)
    subpops = [pop1, pop2, pop3]

    # Run dynamics and create plots
    (average_risks_subpop, average_risks_learner, all_risks, all_thetas, all_alphas, all_prices, all_profits) = run_experiment_with_prices(T, subpops, min_fn=min_fn, early_stop=False)
    
    return(average_risks_subpop, average_risks_learner, all_risks, all_thetas, all_alphas, all_prices, all_profits)


In [5]:
#allocations =  np.array([[0.496, 0.511, 0.498], [0.504, 0.489, 0.502]])
#allocations =  np.array([[0.3, 0.3, 0.3], [0.3, 0.3, 0.3], [0.4, 0.4, 0.4]])
allocations =  np.array([[0.34, 0.33, 0.33], [0.33, 0.34, 0.33], [0.33, 0.33, 0.34]])
#allocations = None
(average_risks_subpop, average_risks_learner, all_risks, all_thetas, all_alphas, all_prices, all_profits) = run_trial(allocations, min_fn = quadratic_min)

Initial Conditions:
There are 3 initial learners
Subpopulation splits: [1, 1, 1]
Initial allocations: [array([0.34, 0.33, 0.33]), array([0.33, 0.34, 0.33]), array([0.33, 0.33, 0.34])]
Optimal decisions theta for each subpop (row-wise) [array([1, 0, 0]), array([0, 1, 0]), array([0, 0, 1])]
----------
t = 0
p: [0, 1.0, 1.0]
α: [[0.381 0.31  0.31 ]
 [0.37  0.32  0.31 ]
 [0.37  0.31  0.32 ]]
θ: [[0.34 0.33 0.33]
 [0.33 0.34 0.33]
 [0.33 0.33 0.34]]
----------
t = 1
p: [0, 2.0, 2.0]
α: [[0.598 0.201 0.201]
 [0.585 0.212 0.203]
 [0.585 0.203 0.212]]
θ: [[0.34  0.33  0.33 ]
 [0.33  0.341 0.33 ]
 [0.33  0.33  0.341]]
----------
t = 2
p: [0, 1.8, 1.8]
α: [[0.843 0.079 0.079]
 [0.833 0.086 0.081]
 [0.833 0.081 0.086]]
θ: [[0.338 0.331 0.331]
 [0.327 0.344 0.329]
 [0.327 0.329 0.344]]
----------
t = 3
p: [0, 1.8, 1.8]
α: [[0.944 0.028 0.028]
 [0.939 0.032 0.029]
 [0.939 0.029 0.032]]
θ: [[0.336 0.332 0.332]
 [0.321 0.351 0.328]
 [0.321 0.328 0.351]]
----------
t = 4
p: [0, 1.62, 1.62]
α: [[0.98  

----------
t = 46
p: [0, 2.1562200000000007, 2.1562200000000007]
α: [[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
θ: [[0.333 0.333 0.333]
 [0.    1.    0.   ]
 [0.    0.    1.   ]]
----------
t = 47
p: [0, 2.1562200000000007, 2.1562200000000007]
α: [[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
θ: [[0.333 0.333 0.333]
 [0.    1.    0.   ]
 [0.    0.    1.   ]]
----------
t = 48
p: [0, 2.1562200000000007, 2.1562200000000007]
α: [[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
θ: [[0.333 0.333 0.333]
 [0.    1.    0.   ]
 [0.    0.    1.   ]]
----------
t = 49
p: [0, 2.1562200000000007, 2.1562200000000007]
α: [[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
θ: [[0.333 0.333 0.333]
 [0.    1.    0.   ]
 [0.    0.    1.   ]]
----------
t = 50
p: [1.0, 2.1562200000000007, 2.1562200000000007]
α: [[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
θ: [[0.333 0.333 0.333]
 [0.    1.    0.   ]
 [0.    0.    1.   ]]
----------
t = 51
p: [2.0, 2.1562200000000007, 2.1562200000000007]
α: [[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
θ: [[0.333 0.333 0.333]
 [0.   

/homes/gws/danji/MultiPlayerRiskReduction/subpop.py:42: RuntimeWarning: invalid value encountered in true_divide
  new_alphas /= np.sum(new_alphas)


LinAlgError: SVD did not converge

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
plt.plot(range(len(all_prices)), all_prices, label="Learner")
all_prices = np.array(all_prices)
#plt.plot(range(len(all_prices)), all_prices - all_prices[:, 0:1], label="Learner")
plt.legend()

In [ ]:
plt.plot(range(len(average_risks_learner)), average_risks_learner, label="Learner")
plt.legend()

In [ ]:
plt.plot(range(len(average_risks_subpop)), average_risks_subpop, label="Subpop")
plt.legend()

In [ ]:
plt.plot(range(len(all_profits)), all_profits, label="Learner")

plt.legend()

In [ ]:
all_alphas = np.array(all_alphas)
plt.plot(range(len(all_alphas)), all_alphas[:, 0, :], label="Subpop")
plt.legend()

In [ ]:
all_alphas = np.array(all_alphas)
plt.plot(range(len(all_alphas[:])), all_alphas[:, :, 0], label="Learner")
plt.legend()